This notebook runs `post_process_visit_lists` to take the outputs of `getDC2VisitList` and produce the per-band AND per-year visit lists for the DDF visits.

Also confirms that stacking the grouped-by-year lists matchs with the full list for each band.

In [1]:
import numpy as np
import pandas as pd
import os
import time

In [2]:
os.chdir('/global/u2/a/awan/LSST/lsstRepos/DC2_visitList/')
!python setup.py install --user

running install
running build
running build_py
running install_lib
running install_egg_info
Removing /global/homes/a/awan/.local/lib/python3.6/site-packages/dc2visitgen-0.1-py3.6.egg-info
Writing /global/homes/a/awan/.local/lib/python3.6/site-packages/dc2visitgen-0.1-py3.6.egg-info


In [3]:
from dc2visitgen import post_process_visit_lists

In [4]:
help(post_process_visit_lists)

Help on function post_process_visit_lists in module dc2visitgen.post_process_visitList:

post_process_visit_lists(dir_to_readfrom, wfd_filename, uddf_filename, base_filetag, outDir, saveData=True, nVisits=None, group_by_year=False, dbpath='/global/cscratch1/sd/awan/dbs_old_unzipped/minion_1016_desc_dithered_v3.db')
    The goal here is to take the all-band visit lists produced by DC2_visitListCode, separate them by band
    and save the obsHistIDs and expMJD corresponding to all the visits in the given band.
    
    Essentially following https://github.com/LSSTDESC/DC2-production/blob/master/Notebooks/survey_design_visit_lists.ipynb
    
    HOWEVER, Run 1.1 uDDF per-band production code include grouping by night, out of which the visits with the minimum airmass
    were chosen. Since all the visits need to be simulated for Run 2 and not just a subset, the uDDF lists are created the same
    way as for WFD: the visits are only grouped by band and ordered by expMJD.



In [5]:
mainDir = '/global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen/'

#### Create the per-band, per-year lists for DDF visits.

In [6]:
# create the per-band lists
nside = 1024
baseDir = '%s/DC2_Run2_Visits_nside%s/'%(mainDir, nside)
out = post_process_visit_lists(dir_to_readfrom=baseDir,
                               wfd_filename=None,
                               uddf_filename='DC2VisitList_minion1016_v3_DDvisits_nside%s_DDvisitsInuDD.csv'%nside,
                               base_filetag='DC2_Run2',
                               saveData=True,
                               group_by_year=True,
                               outDir='%s/visitLists/per_band_per_year_DD_lists'%(baseDir),
                               nVisits=None
                        )

Working with lsst.sims.maf.__version__: 2.9.0.sims

## Working with uDDF

All-band array: shape = (23160, 2)

Outdir: /global/homes/a/awan/LSST/lsstRepos/DC2_visitList/DC2visitGen//DC2_Run2_Visits_nside1024//visitLists/per_band_per_year_DD_lists

------------------------------------------------------------------------
Working with g-band
1911 visits for g-band.
220 visits for year 1.


/global/homes/a/awan/.local/lib/python3.6/site-packages/dc2visitgen/post_process_visitList.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  subset['year'] = [int(np.floor(f/365.25))+1 for f in subset['night']]



Saved DC2_Run2_g-band_Y1_uDDFvisits.txt.

160 visits for year 2.

Saved DC2_Run2_g-band_Y2_uDDFvisits.txt.

190 visits for year 3.

Saved DC2_Run2_g-band_Y3_uDDFvisits.txt.

180 visits for year 4.

Saved DC2_Run2_g-band_Y4_uDDFvisits.txt.

230 visits for year 5.

Saved DC2_Run2_g-band_Y5_uDDFvisits.txt.

160 visits for year 6.

Saved DC2_Run2_g-band_Y6_uDDFvisits.txt.

180 visits for year 7.

Saved DC2_Run2_g-band_Y7_uDDFvisits.txt.

210 visits for year 8.

Saved DC2_Run2_g-band_Y8_uDDFvisits.txt.

201 visits for year 9.

Saved DC2_Run2_g-band_Y9_uDDFvisits.txt.

180 visits for year 10.

Saved DC2_Run2_g-band_Y10_uDDFvisits.txt.

------------------------------------------------------------------------
Working with i-band
3818 visits for i-band.
440 visits for year 1.

Saved DC2_Run2_i-band_Y1_uDDFvisits.txt.

320 visits for year 2.

Saved DC2_Run2_i-band_Y2_uDDFvisits.txt.

380 visits for year 3.

Saved DC2_Run2_i-band_Y3_uDDFvisits.txt.

360 visits for year 4.

Saved DC2_Run2_i-band_

#### Confirm that stacking the by-year lists gives back the all-years list for each band.

In [7]:
per_yr_files = [f for f in os.listdir('%s/visitLists/per_band_per_year_DD_lists/'%(baseDir)) if f.__contains__('Y')]
per_band_files = [f for f in os.listdir('%s/visitLists/per_band_lists/'%(baseDir)) if not f.__contains__('WFD')]

In [8]:
for file in per_band_files:
    print('----------------------------------------------------------------')
    print('Read in %s'%file)
    all_yrs = pd.read_csv('%s/visitLists/per_band_lists/%s'%(baseDir, file), delimiter=' ')
    band = file.split('_')[2]
    print(band)
    subset = [f for f in per_yr_files if f.__contains__(band)]
    
    # run over all the years
    for i, yr in enumerate(range(10)):
        file_yrs = [f for f in subset if f.__contains__('Y%s'%(yr+1))]
        if yr==0:
            file_yr = [f for f in file_yrs if not f.__contains__('Y10')][0]
        else:
            file_yr = file_yrs[0]
        print('Read in %s'%file_yr)
        temp = pd.read_csv('%s/visitLists/per_band_per_year_DD_lists/%s'%(baseDir, file_yr), delimiter=' ').values
        if (i==0):
            all_yrs_recons = temp
        else:
            all_yrs_recons = np.vstack([all_yrs_recons, temp])

    all_yrs_recons = pd.DataFrame(all_yrs_recons, columns=['obsHistID', 'expMJD'])
    
    print('\nFull data array: shape: ', np.shape(all_yrs))
    print('Stacked from yr-arrays: shape: ', np.shape(all_yrs_recons))
    print('\n## Arrays match: %s\n'%np.unique(all_yrs == all_yrs_recons))

----------------------------------------------------------------
Read in DC2_Run2_u-band_uDDFvisits.txt
u-band
Read in DC2_Run2_u-band_Y1_uDDFvisits.txt
Read in DC2_Run2_u-band_Y2_uDDFvisits.txt
Read in DC2_Run2_u-band_Y3_uDDFvisits.txt
Read in DC2_Run2_u-band_Y4_uDDFvisits.txt
Read in DC2_Run2_u-band_Y5_uDDFvisits.txt
Read in DC2_Run2_u-band_Y6_uDDFvisits.txt
Read in DC2_Run2_u-band_Y7_uDDFvisits.txt
Read in DC2_Run2_u-band_Y8_uDDFvisits.txt
Read in DC2_Run2_u-band_Y9_uDDFvisits.txt
Read in DC2_Run2_u-band_Y10_uDDFvisits.txt

Full data array: shape:  (4963, 2)
Stacked from yr-arrays: shape:  (4963, 2)

## Arrays match: [ True]

----------------------------------------------------------------
Read in DC2_Run2_z-band_uDDFvisits.txt
z-band
Read in DC2_Run2_z-band_Y1_uDDFvisits.txt
Read in DC2_Run2_z-band_Y2_uDDFvisits.txt
Read in DC2_Run2_z-band_Y3_uDDFvisits.txt
Read in DC2_Run2_z-band_Y4_uDDFvisits.txt
Read in DC2_Run2_z-band_Y5_uDDFvisits.txt
Read in DC2_Run2_z-band_Y6_uDDFvisits.txt


Okay good, things are working.